In [23]:
## imports
import pandas as pd
import numpy as np
from pulp import *

In [24]:
##Load datasets
Household_dataset=pd.read_csv('SampleOptimizationHouseholdData_02142024.csv')
Intervention_dataset=pd.read_csv('SampleOptimizationInterventionData_02142024.csv')


In [25]:
## Data cleaning and conversions
Household_dataset['intervention_eligibility_start'] = pd.to_datetime(Household_dataset['intervention_eligibility_start'])
Household_dataset['intervention_eligibility_end'] = pd.to_datetime(Household_dataset['intervention_eligibility_end'])
Intervention_dataset['MoveInDate'] = pd.to_datetime(Intervention_dataset['MoveInDate'])


In [26]:
household_ids = Household_dataset['ID'].unique()
intervention_types = ['RRH', 'TSH', 'PSH', 'NOTX']
# Initialize the problem
problem = LpProblem("Maximize_exiting_homelessness", LpMaximize)

# Decision variables
x = LpVariable.dicts("assignment", (household_ids, intervention_types), cat='Binary')


In [27]:
# Objective function
problem += lpSum([x[household_id][intervention_type] * Household_dataset.set_index('ID').at[household_id, 'P_' + intervention_type]
                  for household_id in household_ids for intervention_type in intervention_types])


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Constraints: One intervention per household
for household_id in household_ids:
    problem += lpSum([x[household_id][intervention_type] for intervention_type in intervention_types]) == 1

# Constraints: Intervention capacity (this is an example, you would need to calculate the actual capacity)
for intervention_type in intervention_types:
    problem += lpSum([x[household_id][intervention_type] for household_id in household_ids]) <= Intervention_dataset[Intervention_dataset['ProjectTypeName'] == intervention_type].shape[0]


In [ ]:
# Solve the problem
problem.solve()

# Check the status of the solution
print("Status:", LpStatus[problem.status])


In [ ]:
# # Initialize an empty list to store results
# results = []

# # Iterate over the problem's variables to collect non-zero assignments
# for v in Problem.variables():
#     if v.varValue is not None and v.varValue > 0:
#         results.append(f"{v.name}: {v.varValue}")

# # Print all results after the loop
# for result in results:
#     print(result)